# Model with eval data

In [1]:
import os
import time
import base64
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, IterableDataset, random_split
import pytorch_lightning as pl
from collections import OrderedDict
from random import randrange
from peewee import *

# Connect to the SQLite database
db = SqliteDatabase('2021-07-31-lichess-evaluations-37MM.db')

# Define the Evaluations model
class Evaluations(Model):
    id = IntegerField()
    fen = TextField()
    binary = BlobField()
    eval = FloatField()

    class Meta:
        database = db

    def binary_base64(self):
        return base64.b64encode(self.binary)

# Connect to the database and print LABEL_COUNT
db.connect()
LABEL_COUNT = 37164639
print(LABEL_COUNT)
eval = Evaluations.get(Evaluations.id == 1)
print(eval.binary_base64())

# Define the EvaluationDataset
class EvaluationDataset(IterableDataset):
    def __init__(self, count, limit):
        self.count = count
        self.limit = limit

    def __iter__(self):
        return self

    def __next__(self):
        idx = randrange(self.limit)
        return self[idx]

    def __len__(self):
        return self.limit

    def __getitem__(self, idx):
        eval = Evaluations.get(Evaluations.id == idx + 1)
        bin = np.frombuffer(eval.binary, dtype=np.uint8)
        bin = np.unpackbits(bin, axis=0).astype(np.single)
        eval.eval = max(eval.eval, -15)
        eval.eval = min(eval.eval, 15)
        ev = np.array([eval.eval]).astype(np.single)
        return {'binary': bin, 'eval': ev}

# Limit the dataset to 20,058 entries
dataset = EvaluationDataset(count=LABEL_COUNT, limit=20058)

# Define the EvaluationModel
class EvaluationModel(pl.LightningModule):
    def __init__(self, learning_rate=1e-3, batch_size=2048, layer_count=10):
        super().__init__()
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        layers = []
        for i in range(layer_count - 1):
            layers.append((f"linear-{i}", nn.Linear(808, 808)))
            layers.append((f"relu-{i}", nn.ReLU()))
        layers.append((f"linear-{layer_count - 1}", nn.Linear(808, 1)))
        self.seq = nn.Sequential(OrderedDict(layers))

    def forward(self, x):
        return self.seq(x)

    def training_step(self, batch, batch_idx):
        x, y = batch['binary'], batch['eval']
        y_hat = self(x)
        loss = F.l1_loss(y_hat, y)
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.learning_rate)

    def train_dataloader(self):
        return DataLoader(dataset, batch_size=self.batch_size, num_workers=0, pin_memory=True, persistent_workers=False)

# Training configuration
configs = [
    {"layer_count": 4, "batch_size": 2048},
]

for config in configs:
    version_name = f'{int(time.time())}-batch_size-{config["batch_size"]}-layer_count-{config["layer_count"]}'
    logger = pl.loggers.TensorBoardLogger("lightning_logs", name="chessml", version=version_name)
    trainer = pl.Trainer(devices=1, accelerator="mps", precision="16", max_epochs=1, logger=logger)
    model = EvaluationModel(layer_count=config["layer_count"], batch_size=config["batch_size"], learning_rate=1e-3)
    
    trainer.fit(model)
    break


Using 16bit None Automatic Mixed Precision (AMP)
/Users/primepi/anaconda3/envs/chess/lib/python3.11/site-packages/torch/amp/grad_scaler.py:131: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type       | Params
------------------------------------
0 | seq  | Sequential | 2.0 M 
------------------------------------
2.0 M     Trainable params
0         Non-trainable params
2.0 M     Total params
3.924     Total estimated model params size (MB)


37164639
b'CAAAAAAAAAAQAAAAAAAAAIEAAAAAAAAAJAAAAAAAAABCAAAAAAAAAADvABAAAAAAAAAAAAAAAAgAAAAAAAAAEAAAAAAAAACBAAAAAAAAACQAAAAAAAAAQgAAAAAAAP8AAAABEz8='


/Users/primepi/anaconda3/envs/chess/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/primepi/anaconda3/envs/chess/lib/python3.11/site-packages/lightning_fabric/utilities/data.py:63: UserWarning: Your `IterableDataset` has `__len__` defined. In combination with multi-process data loading (when num_workers > 1), `__len__` could be inaccurate if each worker is not configured independently to avoid having duplicate data.
  rank_zero_warn(
/Users/primepi/anaconda3/envs/chess/lib/python3.11/site-packages/pytorch_lightning/trainer/trainer.py:1609: PossibleUserWarning: The number of training batches (10) is smaller than the logging interval Trainer(log_every_n_st

Epoch 0:   0%|          | 0/10 [00:00<?, ?it/s] 

/Users/primepi/anaconda3/envs/chess/lib/python3.11/site-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


Epoch 0: 100%|██████████| 10/10 [00:02<00:00,  3.79it/s, loss=4.57, v_num=nt-4]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 10/10 [00:02<00:00,  3.75it/s, loss=4.57, v_num=nt-4]


# Model with no eval data

In [3]:
import os
import time
import numpy as np
import torch
import chess
import chess.pgn
import pandas as pd
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import pytorch_lightning as pl
from collections import OrderedDict

# Define the GamesDataset class
class GamesDataset(Dataset):
    def __init__(self, csv_file):
        self.games_df = pd.read_csv(csv_file)
        self.games_df['outcome'] = self.games_df['winner'].map({'white': 2, 'black': 0, 'draw': 1})  # Map results to integers

    def __len__(self):
        return len(self.games_df)

    def __getitem__(self, idx):
        row = self.games_df.iloc[idx]
        moves = row['moves']
        outcome = row['outcome']
        try:
            fen = self.moves_to_fen(moves)
        except Exception as e:
            print(f"Error converting moves to FEN: {e}")
            return self.__getitem__((idx + 1) % len(self.games_df))  # Retry with the next item
        board = chess.Board(fen)
        binary_board = self.board_to_binary(board)
        return {'binary': binary_board, 'outcome': torch.tensor(outcome, dtype=torch.long)}

    def moves_to_fen(self, moves):
        game = chess.pgn.Game()
        node = game
        board = chess.Board()
        for move in moves.split():
            try:
                move_obj = board.parse_san(move)
                board.push(move_obj)
                node = node.add_main_variation(move_obj)
            except ValueError as e:
                raise Exception(f"Invalid move: {move}, Error: {e}")
        return board.fen()

    def board_to_binary(self, board):
        # Convert board to a binary format suitable for NN input
        binary = []
        for square in chess.SQUARES:
            piece = board.piece_at(square)
            if piece:
                binary.extend(self.piece_to_binary(piece))
            else:
                binary.extend([0] * 12)  # 12 channels for empty squares
        if len(binary) != 768:
            print(f"Error: Binary board representation has incorrect length {len(binary)}")
        return torch.tensor(binary, dtype=torch.float)

    def piece_to_binary(self, piece):
        # 12 binary channels for each piece type and color
        piece_map = {
            'P': 0, 'N': 1, 'B': 2, 'R': 3, 'Q': 4, 'K': 5,
            'p': 6, 'n': 7, 'b': 8, 'r': 9, 'q': 10, 'k': 11
        }
        binary = [0] * 12
        binary[piece_map[piece.symbol()]] = 1
        return binary

# Define the OutcomeModel class
class OutcomeModel(pl.LightningModule):
    def __init__(self, learning_rate=1e-3, batch_size=1024, layer_count=10):
        super().__init__()
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        layers = []
        layers.append(('flatten', nn.Flatten()))  # Add flatten layer to flatten the input
        layers.append((f"linear-0", nn.Linear(768, 808)))
        layers.append((f"relu-0", nn.ReLU()))
        for i in range(1, layer_count - 1):
            layers.append((f"linear-{i}", nn.Linear(808, 808)))
            layers.append((f"relu-{i}", nn.ReLU()))
        layers.append((f"linear-{layer_count - 1}", nn.Linear(808, 3)))  # 3 output classes for win, lose, draw
        self.seq = nn.Sequential(OrderedDict(layers))

    def forward(self, x):
        return self.seq(x)

    def training_step(self, batch, batch_idx):
        x, y = batch['binary'], batch['outcome']
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.learning_rate)

    def train_dataloader(self):
        dataset = GamesDataset(csv_file='games.csv')
        return DataLoader(dataset, batch_size=self.batch_size, num_workers=0, pin_memory=True, persistent_workers=False)

# Training configuration
configs = [
    {"layer_count": 4, "batch_size": 512},
    # {"layer_count": 6, "batch_size": 1024},
]

for config in configs:
    version_name = f'{int(time.time())}-batch_size-{config["batch_size"]}-layer_count-{config["layer_count"]}'
    logger = pl.loggers.TensorBoardLogger("lightning_logs", name="chessml", version=version_name)
    trainer = pl.Trainer(devices=1, accelerator="mps", precision="16", max_epochs=1, logger=logger)
    model = OutcomeModel(layer_count=config["layer_count"], batch_size=config["batch_size"], learning_rate=1e-3)
    
    # Uncomment below if you want to find optimal learning rate
    # lr_finder = trainer.tuner.lr_find(model, min_lr=1e-6, max_lr=1e-3, num_training=25)
    
    # Plot the learning rate finder results
    # fig = lr_finder.plot(suggest=True)
    # fig.show()
    
    # Set the suggested learning rate
    # new_lr = lr_finder.suggestion()
    # model.learning_rate = new_lr
    
    trainer.fit(model)
    break


Using 16bit None Automatic Mixed Precision (AMP)
/Users/primepi/anaconda3/envs/chess/lib/python3.11/site-packages/torch/amp/grad_scaler.py:131: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type       | Params
------------------------------------
0 | seq  | Sequential | 1.9 M 
------------------------------------
1.9 M     Trainable params
0         Non-trainable params
1.9 M     Total params
3.862     Total estimated model params size (MB)
/Users/primepi/anaconda3/envs/chess/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus 

Epoch 0:   0%|          | 0/40 [00:00<?, ?it/s] 

/Users/primepi/anaconda3/envs/chess/lib/python3.11/site-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


Epoch 0: 100%|██████████| 40/40 [00:16<00:00,  2.38it/s, loss=0.634, v_num=nt-4]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 40/40 [00:16<00:00,  2.38it/s, loss=0.634, v_num=nt-4]
